# Multiple Threads

In [11]:
import threading
import time

def yes():
    for _ in range(3):
        print("Yes!")
        time.sleep(0.1)
def no():
    for _ in range(3):
        print("       No!")
        time.sleep(0.1)

yes_kid = threading.Thread(target=yes)
no_kid = threading.Thread(target=no)
yes_kid.start()
no_kid.start()
yes_kid.join()
no_kid.join()
print("I will stop this car!")

Yes!
       No!
       No!
Yes!
       No!
Yes!
I will stop this car!


## Passing Data Between Threads

In [37]:
import threading

class Conduit():
    def __init__(self):
        self.datum = None
        
    def put(self,nn):
        self.datum = nn
        
    def get(self):
        return self.datum

def producer(cc):
    for letter in "Pie":
        print(f"sent {letter}")
        cc.put(letter)
    cc.put(None)

def consumer(cc):
    msg = ""
    while True:
        dd = cc.get()
        if dd is None:
            break
        msg = msg+dd
    print("Msg is:", msg)
            
cc = Conduit()
pthread = threading.Thread(target=producer, args=(cc,))
cthread = threading.Thread(target=consumer, args=(cc,))
pthread.start()
cthread.start()

sent P
sent i
sent e
Msg is: 


In [38]:
import threading
import queue


def producer(cc):
    for letter in "Pie":
        print(f"sent {letter}")
        cc.put(letter)
    cc.put(None)

def consumer(cc):
    msg = ""
    while True:
        dd = cc.get()
        if dd is None:
            break
        msg = msg+dd
    print("Msg is:", msg)
            
cc = queue.Queue()
pthread = threading.Thread(target=producer, args=(cc,))
cthread = threading.Thread(target=consumer, args=(cc,))
pthread.start()
cthread.start()

sent P
sent i
sent e
Msg is: Pie


## Using Thread Conditions to Block

The race car example

In [ ]:
from pyuvm import Singleton
import threading

class FlagHolder(metaclass=Singleton):
    def __init__(self, cond=None):
        self.flag = False
        self.cond = cond
        self.set_count = 1

    def set_flag(self, flag):
        print(f"\nSetting flag to {flag}")
        self.set_count += 1
        self.flag = flag
        with self.cond:
            self.cond.notify_all()

def check_flag():
    print(f"Checking {FlagHolder().set_count}", end=" ")
    return FlagHolder().flag

def racecar(number, flag_cond):
    with flag_cond:
        flag_cond.wait_for(check_flag)
    print(f"# {number:02d} Go!", end=" ")

flag_cond = threading.Condition()
flag_holder = FlagHolder(flag_cond)
thread_list=[]

for ii in range(10,13):
    thread = threading.Thread(target=racecar, args=[ii, flag_cond])
    thread_list.append(thread)
for tt in thread_list:
    tt.start()


flag_holder.set_flag(False)
time.sleep(0.1)
flag_holder.set_flag(False)
time.sleep(0.1)
flag_holder.set_flag(True)

In [14]:
import threading
import random
import time

class GuessingGame:
    def __init__(self, answer):
        assert(answer in set(range(1,6))), "Must pick a number between 1 and 5"
        self.guesses = list(range(1,6))
        self.answer = answer
        self.guess = None
        self.guess_cond = threading.Condition()
        self.done = False
        
    def guesser(self):
        while not self.done:
            self.guess = random.choice(self.guesses)
            self.guesses.remove(self.guess)
            print(f"Guessing {self.guess}")
            with self.guess_cond:
                self.guess_cond.notify_all()
            time.sleep(0.1)
            
    def is_it(self):
        print(f"Is it {self.guess}?")
        return self.answer == self.guess
    
    def checker(self):
        with self.guess_cond:
            self.guess_cond.wait_for(self.is_it)
        print(f"Correct: It is {self.guess}!")
        self.done = True

    def play(self):
        print("Play")
        guesser_thread = threading.Thread(target=self.guesser)
        checker_thread = threading.Thread(target=self.checker)
        guesser_thread.start()
        checker_thread.start()
        guesser_thread.join()
        checker_thread.join()

game = GuessingGame(5)
game.play()

Play
Guessing 1
Is it 1?
Guessing 2
Is it 2?
Guessing 4
Is it 4?
Guessing 3
Is it 3?
Guessing 5
Is it 5?
Correct: It is 5!


In [ ]:
None != 5

## Ending Threads
### Terminating Data

In [ ]:
import threading

def producer(numb, qq):
    for nn in range(numb):
        qq.put(nn)
        print(f"sent {nn}", end=" ")
    qq.put(None)

def consumer(qq):
    nn = qq.get()
    while nn is not None:
        print(f"got {nn}", end=" ")
        nn = qq.get()

queue = Queue(maxsize=1)
pthread = threading.Thread(target=producer, args=[4, queue])
cthread = threading.Thread(target=consumer, args=[queue])
pthread.start()
cthread.start()


### Termination Flag

In [3]:
import threading
from pyuvm import Singleton
import time
from queue import Queue, Empty

class TerminateExample():
    def __init__(self):
        self.finished = False
        self.qq = Queue(maxsize=1)
        
    def producer(self,numb):
        for nn in range(numb):
            print(f"sent {nn}", end=" ")
            self.qq.put(nn)
            time.sleep(0.2)
        self.finished = True

    def consumer(self):
        while not self.finished:
            try:
                nn = self.qq.get(timeout=0.1)
                time.sleep(0.2)
                print(f"got {nn}", end=" ")
            except Empty:
                pass

    def run(self):
        pthread = threading.Thread(target=self.producer, args=(4,))
        cthread = threading.Thread(target=self.consumer)
        pthread.start()
        cthread.start()
        cthread.join()
        pthread.join()

TerminateExample().run()

sent 0 got 0sent 1  sent 2got 1  got 2 sent 3 got 3 